In [ ]:
# Check Folders items
import os 
import os
import re
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

#Check Folder find how many files csv and excel in the folder in which this file is running
def checkfolder():
    files = os.listdir()
    print('Number of items in this folder :', len(files))
    b, csv, xlsx =1, [], []
    for file in os.listdir():
        print(b,file)
        b+=1
    print()
    for file in os.listdir():
        if file.endswith('.csv'):
            csv.append(file)
        if file.endswith(".xls") or file.endswith(".xlsx") or file.endswith(".xlsm"):
            xlsx.append(file)
            
    print('There are '+str(len(csv))+' csv files :',csv)
    print('There are '+str(len(xlsx))+' excel files :',xlsx)
#Create list of csv and xlsx file 
for file in os.listdir():
    csv = [ file for file in os.listdir() if file.endswith('.csv') ] # Check .csv file in folder and create list
    xlsx= [ file for file in os.listdir() if file.endswith(".xls") or file.endswith(".xlsx") or file.endswith(".xlsm")] # Check .xlsx file in folder and create list
df1, df2 = pd.DataFrame(), pd.DataFrame() # create empty dataframe as master dataframe
for i in csv:
    df = pd.read_csv(i)
    df1 = df1.append(df) # Append dataframes to master dataframe df1 for csv file
for i in xlsx:
    df = pd.read_excel(i, sort=False)
    df2 = df2.append(df) # Append dataframes to master dataframe df2 for excel file

In [ ]:
# Check header function check given file contain header row or not
def check_header(df): 
    flag = 0
    deCol = ['customer','name','date','gender','time','id','address','number','mobile','contact','phone'] # for Example columns names list
    for column in list(df.columns):
        if str(column).lower() in deCol or str(column).split()[0].lower() in deCol:
            flag=1
            break;
    if flag == 0:
        print("The file is missing the header row")
    else:
        print("The file has header row")
    return flag

def dfNotHeader():                            # Create DF for which file has not Header 
    df1, df2 = pd.DataFrame(), pd.DataFrame() # create empty dataframe as master dataframe
    for i in csv:
        df = pd.read_csv(i, header=None, prefix='X')
        df1 = df1.append(df)   # Append dataframes to master dataframe df1 for csv file
#         print('CSV master file save in df1')

    for i in xlsx:
        df = pd.read_excel(i, header=None, prefix='X')
        df2 = df2.append(df)  # Append dataframes to master dataframe df2 for excel file
#         print('XLSX master file save in df2')
    return df1,df2

def dfWithHeader():                               # Create DF for which file has Header
    data1, data2 = pd.DataFrame(), pd.DataFrame() # create empty dataframe as master dataframe
    for i in csv:
        df = pd.read_csv(i)
        data1 = data1.append(df, sort=False)   # Append dataframes to master dataframe data1 for csv file
#         print('CSV master file save in data1')

    for i in xlsx:
        df = pd.read_excel(i)
        data2 = data2.append(df)  # Append dataframes to master dataframe data2 for excel file
#         print('XLSX master file save in data2')
    return data1,data2 

def header_pre():
    # Check csv or xlsx file present in folder and they have header or not
    global ch1,ch2
    if len(csv)!=0:
        print('\nCSV files present in Folder')
        a = pd.read_csv(csv[0])
        ch1 = check_header(a)                   # Ch1 is flag if header is present ch1 is 1 otherwise 0
    else:
        ch1 = 0
        print('No CSV files present in Folder')
    if len(xlsx)!=0:
        print('\nExcel files present in Folder')
        b = pd.read_excel(xlsx[0])
        ch2 = check_header(b)                  # Ch2 is flag if header is present ch2 is 1 otherwise 0
    else:
        ch2 = 0
        print('No Excel files present in Folder')
        
def create_master_file():
    global data1,data2,df1,df2
    if ch1==1:
        print('CSV file having Header master file store in data1')
        data1 = dfWithHeader()[0]
    elif ch2==1:
        print('Excel file having Header master file store in data2')
        data2 = dfWithHeader()[1]
    elif ch1==0:
        print('CSV file not having Header master file store in df1')
        df1 = dfNotHeader()[0]
    if ch2==0:
        print('Excel file not having Header master file store in df2')
        df2 = dfNotHeader()[1]
    else:
        print()

# create data for file contain header row
def create_dataframe(df):
    data = pd.DataFrame(columns=['Customer_Name','Mobile_Number'])
    for column in df.columns:
        try:
            if ('customer' in str(column).lower()):
                data['Customer_Name'] = df[column]

            elif ('mobile' in str(column).lower() or 'contact' in str(column).lower()):
                data['Mobile_Number'] = df[column]
        
        except NameError:
            print("There are no columns with field either <Customer> or <Mobile / Contact>")
            print("Please check the header row and rename the column appropriately")
        
    return data

def create_dataframe2(f,i,j):
    if f=='df2':
        df = df2.iloc[:,[i,j]]
        df = df.set_index(np.arange(len(df)))
    elif f=='df1':
        df = df1.iloc[:,[i,j]]
        df = df.set_index(np.arange(len(df)))    
    return df

def see_dataframe(f):
    if f=='df2':
        return df2.head()
    elif f=='df1':
        return df1.head()  
    
# clean no function clean data in dataset only focus on mobile number and name column and created flag store in new column
def clean_no():
    def sortNumber10(a):                         #Number with 10 length of string 
        return a[:10]

    def sortNumber20(a):                         #Number with 20 length of string 
        return a[:10],a[10:20]

    def phone(s):                                # Function for checkng Landline No
        Pattern = re.compile(r'011|022|056|052') # Pattern check  for Starting with given STD codes if any missing STD code add it
        match=re.match(Pattern,s)
        if match:
            return s
        else:
            return ''

    def InValidNo(s):                         # Function for checking Invalid Number
        Pattern1 = re.compile(r'1|2|3|4|5|6') # Pattern check  for Starting with number
        Pattern2 = re.compile(r'7|8|9')       # Number starting with Number
        if re.match(Pattern1,s):
            return s
        elif re.match(Pattern2,s):
            if len(s)<=9 or len(s)>11 and len(s)<20: # Wrong length of digit
                return s
        return ''

    df['X5'] = df['X4'].apply(lambda x: re.sub('[^0-9]+','',str(x)))                             # Remove character and special character
    df['X6'] = df['X5'].apply(lambda x : sortNumber20(x)[0] if len(x)==20 else sortNumber10(x) ) # 1st Mobile number store when two number is there
    df['X7'] = df['X5'].apply(lambda x : sortNumber20(x)[1] if len(x)==20 else '') # 2nd Mobile number store when two number is there
    df['X8'] = df['X5'].apply(phone)                                               # Landline No...
    df['X9'] = df['X5'].apply(InValidNo)                                           # Invalid Number Logically

    duName = (df[df.duplicated(['X2'])]['X2'].values).tolist()                     # create list for duplicate name
    df['X3'] = df['X2'].apply(lambda x: 'DuplicatName' if x in duName  else '')    # create flag for duplicate names

    duNo = (df[df.duplicated(['X6'])]['X6'].values).tolist()
    duNo = [x for x in duNo if str(x)!= '']
    df['X66'] = df['X6'].apply(lambda x: 'DuplicatNumber' if x in duNo  else ' ')  # create flag for duplicate number

    duNo2 = (df[df.duplicated(['X7'])]['X7'].values).tolist()
    duNo2 = [x for x in duNo2 if str(x)!= '']
    df['X77'] = df['X7'].apply(lambda x: 'DuplicatNumber' if x in duNo2  else ' ') # create flag for duplicate number2

    duNo3 = (df[df.duplicated(['X8'])]['X8'].values).tolist()
    duNo3 = [x for x in duNo3 if str(x)!= '']
    df['X88'] = df['X8'].apply(lambda x: 'DuplicatNumber' if x in duNo3  else ' ') # create flag for duplicate number2
    # rename default column name to actual names
    final1 = df.rename(columns={'X2':'Name','X3':'Duplicate_name','X4':'Phone_Number','X5':'Numbers','X6':'Phone_no1','X66':'DuplicatePhone_no1','X7':'Phone_no2','X77':'DuplicatePhone_no2','X8':'Landline_no','X88':'DuplicateLL_no','X9':'Invalid_no'})
    col = ['Name','Duplicate_name', 'Phone_Number','Numbers', 'Phone_no1','DuplicatePhone_no1', 'Phone_no2','DuplicatePhone_no2','Landline_no','DuplicateLL_no','Invalid_no']
    final = final1[col]
    return final

# Return dataframe with number of counts
def number_count():
    #getting count for new series
    import warnings
    warnings.filterwarnings('ignore')
    def nscount(x):
        pattern=re.compile("^[7-8]")
        match=re.match(pattern,x)
        if match:
            return 1
        else:
            return 0

    nl1=data['Phone_no1'].apply(nscount)
    nl2=data['Phone_no2'].apply(nscount)
    data['New_Series_Count']=nl1+nl2

    #getting count for old series
    def oscount(x):
        pattern=re.compile("^9")
        match=re.match(pattern,x)
        if match:
            return 1
        else:
            return 0

    ol1=data['Phone_no1'].apply(oscount)
    ol2=data['Phone_no2'].apply(oscount)
    data['Old_Series_Count']=ol1+ol2

    #getting count for landline numbers
    def land_line(x):
        if len(x)!=0:
            return 1
        else:
            return 0

    data['Landline'] = data['Landline_no'].apply(land_line)
    return data

# return datfreame saperate number with new/old series and landline number
def Create_checkNo():
    def checkNo(a):
        if re.match(re.compile("^9"),a):
            return 'OldNo'
        elif re.match(re.compile("^[7-8]"),a):
            return 'NewNo'
        elif re.match(re.compile("^[0-4]"),a):
            return 'LNo'
        else:
            return ''
    data['CheckSeries1'] = data['Phone_no1'].apply(lambda x: checkNo(x))
    data['CheckSeries2'] = data['Phone_no2'].apply(lambda x: checkNo(x))
    data['Landline'] = data['Landline_no'].apply(lambda x: checkNo(x))
    
    return data

In [ ]:
print('Check Folder Details:')  
checkfolder()  # check folder and create list for seperate format of files. e.g.- CSV file list
print('*'*100)
header_pre()
print('*'*100)
create_master_file()
print('*'*100)
print('Given Data files contains header row (y/n) :')
yn = input()
print('*'*100)

if yn == 'y':
    print('Enter Master file name :')
    f = input()
    print('*'*100)
    global data
    if f=='data1':
        data = create_dataframe(data1)
    if f=='data2':
        data = create_dataframe(data2)
    if f=='df1':
        data = create_dataframe(d1)
    if f=='df2':
        data = create_dataframe(df2)
    # data.head()
    col = data.columns
    df = data.rename(columns={col[0]:'X2',col[1]:'X4'})
    print(df.head())
else:
    print("Your Given Data has not columns name. So See follow Dataset and please give follow info for further process....")
    print('Enter Master file name :')
    f = input()
    dataframe = see_dataframe(f)
    print(dataframe)
    print('Enter column number for Name :')
    i = int(input())
    print('Enter column number for Phone Number :')
    j = int(input())
    print('*'*100)
    data = create_dataframe2(f,i,j)
    col = data.columns
    df = data.rename(columns={col[0]:'X2',col[1]:'X4'})
    print(df.head())

# Clean data, find duplicate number and name and create flag for each

In [ ]:
global data
# data = clean_no()
# data.head()
col = ['Name','Duplicate_name','Phone_no1','DuplicatePhone_no1', 'Phone_no2','DuplicatePhone_no2','Landline_no','DuplicateLL_no','Invalid_no']
a = clean_no()
data = a[col]
data.head()

# Seperate Number landline number and invalid number

In [ ]:
col = ['Name','Phone_no1', 'Phone_no2','Landline_no','Invalid_no']
a = clean_no()
data = a[col]
data.head()

# Count new, old and landline numbers

In [ ]:
col = ['Name','Phone_no1', 'Phone_no2','Landline_no']
data = data[col]
data = number_count()
data.head()

# check number is old series or new series number or landline number

In [ ]:
col = ['Name','Phone_no1', 'Phone_no2','Landline_no']
data = data[col]
data = Create_checkNo()
col = ['Name', 'Phone_no1', 'CheckSeries1', 'Phone_no2','CheckSeries2', 'Landline_no', 'Landline']
data = data[col]
data.head()